In [69]:
import numpy as np
import pandas as pd

pd.set_option("display.width", 1000)

In [70]:
logs = [
    "../log-simple-cloudflare",
    "../log-single-cloudflare",
    "../log-simple-client1",
    "../log-single-client1",
    # "../log-multi-client1",
    # "../log"
]


In [71]:
def analyze(df: pd.DataFrame) -> float:
    df = df[
        ((df["protocol"] == "h2") | (df["protocol"] == "h3")) & (df["status"] == 200)
    ]
    # print(f"{df.size=}")

    df_groupby = df[["n", "startMili", "endMili", "baseUnixTime"]].groupby("n")

    print(f'start = {df_groupby.min()["baseUnixTime"].min()}')
    print(f'end = {df_groupby.max()["baseUnixTime"].max()}')

    df_groupby_count = df_groupby.count()["startMili"]
    df_success = df_groupby_count == df_groupby_count.mode()[0]
    # print(f"{df_success[df_success].size=}")

    df_mins = df_groupby.min()["startMili"]
    df_maxs = df_groupby.max()["endMili"]
    duration_df = df_maxs - df_mins
    duration_df = duration_df[df_success]

    duration_df_describe = duration_df.describe()

    df_lower_limit = duration_df_describe["mean"] - duration_df_describe["std"] * 3
    df_upper_limit = duration_df_describe["mean"] + duration_df_describe["std"] * 3

    duration_df_filtered = duration_df[
        (df_lower_limit < duration_df) & (duration_df < df_upper_limit)
    ]

    df_des = duration_df_filtered.describe()

    return pd.DataFrame([duration_df_describe, df_des], index=["prefilterd", "filterd"])


In [72]:
def compare(log: str):
    h2 = pd.read_json(log + "/h2-performances-processed.json")
    h3 = pd.read_json(log + "/h3-performances-prosessed.json")
    # print()
    print(log)
    print("h2")
    result_h2 = analyze(h2)
    print(result_h2)
    print()
    print("h3")
    result_h3 = analyze(h3)
    print(result_h3)

    print()


In [73]:
for log in logs:
    compare(log)

../log-simple-cloudflare
h2
start = 1675310652918
end = 1675311973540
             count        mean        std   min   25%    50%    75%    max
prefilterd  1000.0  109.227000  15.950043  84.0  99.0  105.0  115.0  217.0
filterd      976.0  107.622951  12.134598  84.0  99.0  104.0  114.0  157.0

h3
start = 1675312575286
end = 1675313875304
             count       mean        std   min   25%   50%   75%    max
prefilterd  1000.0  87.078000  12.262475  66.0  80.0  84.0  90.0  198.0
filterd      984.0  86.227642  10.135953  66.0  80.0  84.0  90.0  123.0

../log-single-cloudflare
h2
start = 1675303474346
end = 1675306119488
            count         mean        std     min     25%     50%     75%     max
prefilterd  998.0  1443.545090  30.278041  1399.0  1426.0  1436.0  1451.0  1758.0
filterd     976.0  1440.561475  21.674854  1399.0  1426.0  1436.0  1450.0  1534.0

h3
start = 1675306722382
end = 1675309408061
            count         mean         std     min     25%     50%      75%     